# Download data

In [ ]:
import importlib

from Project.data.VCTK import VCTKDataset
# importlib.reload(Project.data.VCTK) # prevent cache when making changes

dataset = VCTKDataset(vctk_dataset_path)
print(len(dataset))
print(dataset[0])